# Basic Decision tree 

import libraries


In [28]:
import numpy as np 
import pandas as pd
import csv

In [29]:
path = pd.read_csv('data/weather_data_50.csv')
path.head()

,Day,Conditions,wind_speed,Umbrella
0,1,Cloudy,15,No
1,2,Sunny,15,Yes
2,3,Cloudy,28,No
3,4,Sunny,9,No
4,5,Sunny,29,Yes


Load and preprocesing data

In [35]:
def load_data_from_csv(path):
    """
    Converts CSV data into numerical form for decision tree.
    """
    X, y = [], []
    condition_mapping = {"sunny": 0, "cloudy": 1, "rainy": 2}

    with open(path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            X.append([condition_mapping[row['conditions']], float(row['wind'])])
            y.append(1 if row['umbrella'] == "yes" else 0)

    return np.array(X), np.array(y)

building the tree and our measures of impurity 
the TreeNodes class represents the decision tree
when the tree expands into two different brances(childnodes)

In [31]:
FEATURES = {0: "conditions", 1: "wind_speed"}

class TreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

def gini_impurity(y):
    if len(y) == 0:
        return 0
    classes, counts = np.unique(y, return_counts=True)
    probabilities = counts / len(y)
    return 1 - np.sum(probabilities ** 2)

def weighted_impurity(left_y, right_y, impurity_function):
    n = len(left_y) + len(right_y)
    if n == 0:  # Prevent division by zero
        return 0
    left_weight = len(left_y) / n
    right_weight = len(right_y) / n
    return left_weight * impurity_function(left_y) + right_weight * impurity_function(right_y)


Iterate over all features and values to find which threshold yeilds the lowest impurity

In [32]:
def find_best_split(X, y, impurity_function):
    best_feature = None
    best_threshold = None
    best_impurity = float('inf')

    for feature_idx in range(X.shape[1]):
        sorted_indices = np.argsort(X[:, feature_idx])
        X_sorted = X[sorted_indices, feature_idx]
        y_sorted = y[sorted_indices]

        for i in range(1, len(X_sorted)):
            if X_sorted[i] == X_sorted[i - 1]:  # Skip identical thresholds
                continue
            threshold = (X_sorted[i] + X_sorted[i - 1]) / 2
            left_y = y_sorted[:i]
            right_y = y_sorted[i:]
            split_impurity = weighted_impurity(left_y, right_y, impurity_function)

            if split_impurity < best_impurity:
                best_feature = feature_idx
                best_threshold = threshold
                best_impurity = split_impurity

    if best_feature is None:
        return None, None, None

    best_feature_word = FEATURES.get(best_feature, f"Feature {best_feature}")
    print(f"\nBest Feature: {best_feature_word}")
    print(f"Best Threshold: {best_threshold}")
    print(f"Best Impurity: {best_impurity}\n")

    return best_feature, best_threshold, best_impurity

To build the tree the process of finding te best split

In [33]:
def build_tree(X, y, impurity_function, depth=0, max_depth=5, min_samples_split=2, min_impurity_decrease=1e-7):
    if len(y) < min_samples_split or depth >= max_depth or impurity_function(y) < min_impurity_decrease:
        leaf_value = np.bincount(y).argmax()
        return TreeNode(value=leaf_value)

    print(f"\n# {depth + 1}st Iteration")  # Print iteration number

    best_feature, best_threshold, best_impurity = find_best_split(X, y, impurity_function)
    if best_feature is None:
        leaf_value = np.bincount(y).argmax()
        return TreeNode(value=leaf_value)

    left_indices = X[:, best_feature] <= best_threshold
    right_indices = X[:, best_feature] > best_threshold

    left_subtree = build_tree(X[left_indices], y[left_indices], impurity_function, depth + 1, max_depth, min_samples_split, min_impurity_decrease)
    right_subtree = build_tree(X[right_indices], y[right_indices], impurity_function, depth + 1, max_depth, min_samples_split, min_impurity_decrease)

    return TreeNode(feature=best_feature, threshold=best_threshold, left=left_subtree, right=right_subtree)


# Test data 

In [38]:
# Example Data
X = np.array([[1, 30], [2, 50], [1, 40], [2, 60], [1, 20], [2, 80], [1, 10], [2, 70]])
y = np.array([0, 1, 0, 1, 0, 1, 0, 1])

# Build Tree
build_tree(X, y, gini_impurity)


# 1st Iteration

Best Feature: conditions
Best Threshold: 1.5
Best Impurity: 0.0



Worked example for Regression

In [40]:
filename = pd.read_csv('data/house_prices.csv')
filename.head()

,House,Size in m²,No of rooms,Price
0,1,50,2,150
1,2,60,3,180
2,3,70,3,190
3,4,80,4,210
4,5,90,4,230


modify the data loading function to ingest the housing data

In [41]:
def load_data_from_csv(filename):
    X, y = [], []
    with open(filename, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            X.append([float(row['size']), float(row['num_rooms'])])
            y.append(float(row['price']))
    return np.array(X), np.array(y)

The TreeNode class remains exactly the same, but since we are now looking at a regression task, instead of classification, the impurity measure will be different — in this occasion: Variance.

In [42]:
def variance(y):
    if len(y) == 0:
        return 0
    return np.var(y)

def weighted_variance(left_y, right_y):
    n = len(left_y) + len(right_y)
    return (len(left_y) / n) * variance(left_y) + (len(right_y) / n) * variance(right_y)

The difference between building a regression tree and a classification tree lies in the leaf nodes and it's really subtle. When you reach the leaf node at a classification tree your output should be one of the possible classes, this is why we use np.bincount(y).argmax(), as this returns the class that appears the most in that final group. 

In [47]:
def build_regression_tree(X, y, impurity_function, depth=0, max_depth=5, min_samples_split=2, min_variance_decrease=1e-7):
    if len(y) < min_samples_split or depth >= max_depth or impurity_function(y) < min_variance_decrease:
        return TreeNode(value=np.mean(y))
    
    best_feature, best_threshold, best_variance = find_best_split(X, y, variance)
    if best_feature is None:
        return TreeNode(value=np.mean(y))
    
    left_indices = X[:, best_feature] <= best_threshold
    right_indices = X[:, best_feature] > best_threshold
    
    left_subtree = build_regression_tree(X[left_indices], y[left_indices], depth + 1, max_depth, min_samples_split, min_variance_decrease)
    right_subtree = build_regression_tree(X[right_indices], y[right_indices], depth + 1, max_depth, min_samples_split, min_variance_decrease)
    
    return TreeNode(feature=best_feature, threshold=best_threshold, left=left_subtree, right=right_subtree)

Test data

In [ ]:
# Example dataset 
X = np.array([
    [50, 2], [60, 3], [70, 3], [80, 4], [90, 4],
    [100, 5], [110, 5], [120, 6], [130, 6], [140, 7]
])
y = np.array([150, 180, 190, 210, 230, 250, 270, 290, 310, 330])

# Build and print the tree
regression_tree = build_regression_tree(X, y, variance)
print(regression_tree)


Best Feature: conditions
Best Threshold: 95.0
Best Impurity: 768.0

